Here I use the base python environment where ipywidgets is installed. I do not dare to change things on the nODE environment anymore

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown

In [ ]:
# Define your function
def f(x, a=1.0, b=0.0):
    return a * x / (b + np.exp(-x))

# Plotting function with adjustable parameters
def plot_func(a=1.0, b=0.0):
    x = np.linspace(-10, 10, 500)
    y = f(x, a, b)
    
    plt.figure(figsize=(6,4))
    plt.plot(x, y, label=f"a={a:.2f}, b={b:.2f}")
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.legend()
    plt.grid(True)
    plt.show()

# Create interactive sliders
interact(plot_func, 
         a=FloatSlider(min=0, max=5, step=0.1, value=1), 
         b=FloatSlider(min=-2, max=2, step=0.1, value=0));

In [ ]:


# --- activations ---
def relu(z):     return np.maximum(0, z)
def tanh(z):     return np.tanh(z)
def sigmoid(z):  return 1 / (1 + np.exp(-z))
def identity(z): return z

ACTS = {
    "ReLU": relu,
    "tanh": tanh,
    "sigmoid": sigmoid,
    "identity": identity,
}

# --- Phi with W1=1, eps=1, delta=1 ---
def Phi(x, W0, b0, b1, d, W1, act_name="tanh"):
    sigma = ACTS[act_name]
    return W1 * (x + d * sigma(W0 * x + b0)) + b1

# --- plotting function ---
def plot_phi(W0=1.0, b0=0.0, b1=0.0, d = 1., W1 = 1., act="tanh", x_min=-5.0, x_max=5.0):
    x = np.linspace(x_min, x_max, 800)
    y = Phi(x, W0, b0, b1, d, W1, act)

    plt.figure(figsize=(6,4))
    plt.plot(x, y, label=r"$\Phi(x)$")
    plt.axhline(0, linewidth=0.8)
    plt.axvline(0, linewidth=0.8)
    plt.xlabel("x")
    plt.ylabel("Φ(x)")
    plt.title(f"Φ(x) = W_1(x + d{act}(W₀ x + b₀)) + b₁")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()

# --- interactive controls ---
interact(
    plot_phi,
    W0=FloatSlider(min=-20, max=5, step=0.1, value=1.0, description="W₀"),
    b0=FloatSlider(min=-5, max=5, step=0.1, value=0.0, description="b₀"),
    b1=FloatSlider(min=-5, max=5, step=0.1, value=0.0, description="b₁"),
    d=FloatSlider(min=0.01, max=5, step=0.1, value=0.0, description="d"),
    W1=FloatSlider(min=-5, max=50, step=0.1, value=0.0, description="W₁"),
    act=Dropdown(options=list(ACTS.keys()), value="tanh", description="σ"),
    x_min=FloatSlider(min=-1, max=0, step=0.5, value=-5.0, description="x min"),
    x_max=FloatSlider(min=0, max=1, step=0.5, value=5.0, description="x max"),
);


In [ ]:
import numpy as np

def Phi_2layers(x, W0, b0, W1, b1, sigma=np.tanh):
    """
    Computes:
        phi(x) = x + sigma(W0 x + b0) + sigma(W1 (x + sigma(W0 x + b0)) + b1) + 1
    with delta = eps = W2 = b2 = 1.

    Parameters
    ----------
    x : scalar or np.ndarray
    W0, W1 : scalar or np.ndarray (use shapes compatible with x for matmul)
    b0, b1 : scalar or np.ndarray (broadcastable to outputs)
    sigma : callable, elementwise activation (default: np.tanh)

    Returns
    -------
    same shape as the affine outputs
    """
    def lin(W, x, b):
        # If W is array-like and x is array-like, use matrix multiply, else fallback to elementwise *
        return W * x + b

    h0 = sigma(lin(W0, x, b0))              # sigma(W0 x + b0)
    inner = x + h0                           # (eps x + delta*sigma(...)) with eps=delta=1
    h1 = sigma(lin(W1, inner, b1))           # sigma(W1 * inner + b1)
    out = inner + (h1 - x) + 1               # equals x + h0 + h1 + 1, written to avoid extra temp
    # Simpler/readable: out = x + h0 + h1 + 1
    return x + h0 + h1 + 1


# --- plotting function ---
def plot_Phi2layers(W0=1.0, b0=0.0, W1 = 1.0, b1=0.0, x_min=-5.0, x_max=5.0):
    x = np.linspace(x_min, x_max, 800)
    y = Phi_2layers(x, W0, b0, W1, b1)
    plt.figure(figsize=(6,4))
    plt.plot(x, y, label=r"$\Phi(x)$")
    plt.axhline(0, linewidth=0.8)
    plt.axvline(0, linewidth=0.8)
    plt.xlabel("x")
    plt.ylabel("Φ(x)")
    plt.title(f"two layer model")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()

# --- interactive controls ---
interact(
    plot_Phi2layers,
    W0=FloatSlider(min=-20, max=5, step=0.1, value=1.0, description="W₀"),
    b0=FloatSlider(min=-5, max=5, step=0.1, value=0.0, description="b₀"),
    W1=FloatSlider(min=-20, max=5, step=0.1, value=1.0, description="W₁"),
    b1=FloatSlider(min=-5, max=5, step=0.1, value=0.0, description="b₁"),
    x_min=FloatSlider(min=-20, max=0, step=0.5, value=-5.0, description="x min"),
    x_max=FloatSlider(min=0, max=20, step=0.5, value=5.0, description="x max"),
);


In [ ]:

def f(x):
    y = -1/x
    return (1/x) * np.arccosh(1/np.sqrt(y))

# domain: x < -1
x = np.linspace(-20, -1.00001, 500)
y = f(x)

plt.figure(figsize=(6,4))
plt.plot(x, y, label=r"$f(x)=\frac{1}{x}(\sigma')^{-1}(-1/x)$")
plt.axhline(0, color='k', lw=0.8)
plt.axvline(0, color='k', lw=0.8)
plt.xlabel("x")
plt.ylabel("f(x)")
plt.title("Plot of f(x) for σ = tanh")
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()
